# Описание задания
Постройте сегментацию изображений облаков типа Fish, используя сети Unet, PSPNet или FPN. В качестве базовых сетей можно использовать ResNet, MobileNet, DenseNet или любые другие подходящие. Можно использовать обученные модели сетей (входные размеры 384х256).

Постройте ансамбль предсказаний, выбирая среднее значение из нескольких. Выгрузите результаты предсказания в требуемом формате (sample_submission.csv).

Данные:
* video.ittensive.com/machine-learning/clouds/train.csv.gz (54 Мб)
* video.ittensive.com/machine-learning/clouds/train_images_small.tar.gz (212 Мб)
* video.ittensive.com/machine-learning/clouds/test_images_small.tar.gz (142 Мб)
* video.ittensive.com/machine-learning/clouds/sample_submission.csv.gz

Модели:
* video.ittensive.com/machine-learning/clouds/unet.fish.h5
* video.ittensive.com/machine-learning/clouds/fpn.fish.h5
* video.ittensive.com/machine-learning/clouds/pspnet.fish.h5

Итоговый файл с кодом (.py или .ipynb) выложите в github с портфолио.

In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
import tensorflow as tf
from segmentation_models.losses import dice_loss
from keras.utils import load_img, img_to_array
from keras import optimizers
from keras import backend as K
import re
import os

os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

TEST_IMAGES_FOLDER = os.path.join(os.getcwd(), "test_images_small")

MODEL_SERIALIZED_FILES = {
    'unet': 'unet.fish.h5',
    'fpn': 'fpn.fish.h5',
#     Эта модель не загружается ValueError: bad marshal data (unknown type code) 'pspnet': 'pspnet.fish.h5'
}

Segmentation Models: using `keras` framework.


In [2]:
batch_size = 10
image_x = 384 # 525
image_y = 256 # 350
image_ch = 3 # 3


def load_test_x(files_list: list):
    x = [[]] * len(files_list)
    for j, file in enumerate(files_list):
        img = load_img(os.path.join(TEST_IMAGES_FOLDER, file), target_size=(image_y, image_x))
        img = img_to_array(img)
        x[j] = np.expand_dims(img, axis=0)
    return np.array(x).reshape(len(files_list), image_y, image_x, image_ch)


def load_test_data(files_list: list, batch_size):
    while True:
        batch_start = 0
        batch_end = batch_size
        while batch_start < len(files_list):
            limit = min(batch_end, len(files_list))
            yield (load_test_x(files_list[batch_start:limit]))
            batch_start += batch_size   
            batch_end += batch_size


def is_image_file(file_name: str) -> bool:
    return os.path.isfile(os.path.join(TEST_IMAGES_FOLDER, file_name)) and (re.match(r'^.+\.jpg$', file_name) is not None)


def calculate_average_predictions(predictions: dict) -> list:
    result_data = list()
    predicted_values = list(predictions.values())
    models_number = len(predicted_values)
    if models_number <= 1:
        return None
    
    for cortege_number in range(len(predicted_values[0])):
        result = predicted_values[0][cortege_number][:, :, 0]
        for model_index in range(1, models_number):
            result += predicted_values[model_index][cortege_number][:, :, 0]
        result_data.append((result / models_number).flatten())
        
    return result_data


def rle_encode(to_encode: np.array) -> list:
    rle_encoded = list()
    start_pixel, pixels_number = -1, -1
    for pixel_index, pixel_value in enumerate(to_encode):
        if pixel_value == 1:
            if start_pixel == -1:
                start_pixel = pixel_index
                pixels_number = 1
            else:
                pixels_number += 1
        else:
            if start_pixel != -1:
                rle_encoded.append(start_pixel)
                rle_encoded.append(pixels_number)
                start_pixel = -1
                pixels_number = -1
    return rle_encoded


def convert_predictions_to_rle_column(predictions: list) -> pd.Series:
    encoded_pixels_column = []
    for cortege in predictions:
        rle_encoded = ' '.join(str(value) for value in rle_encode(cortege.astype("int8")))
        if len(rle_encoded) > 0:
            encoded_pixels_column.append(rle_encoded)
        else:
            encoded_pixels_column.append(None)
     
    return pd.Series(encoded_pixels_column)

### Тестовые данные

In [3]:
test_files_list = [
    file_name for file_name in os.listdir(TEST_IMAGES_FOLDER) if is_image_file(file_name)
]
test_files_list[:5]

['002f507.jpg', '0035ae9.jpg', '0038327.jpg', '004f759.jpg', '005ba08.jpg']

### Загрузка моделей и построение предсказаний

In [4]:
%%time
predictions = dict()
for model_name, model_file in MODEL_SERIALIZED_FILES.items():
    print("Работа с моделью", model_name)
    model = keras.models.load_model(model_file, compile=False)
    predictions[model_name] = model.predict(load_test_data(test_files_list, 1), steps=len(test_files_list), verbose=1)
    del model
    print()

Работа с моделью unet
3699/3699 [==============================] - 847s 229ms/step

Работа с моделью fpn
3699/3699 [==============================] - 744s 201ms/step

CPU times: total: 4h 28min 14s
Wall time: 26min 39s


In [5]:
%%time
data = calculate_average_predictions(predictions)

CPU times: total: 891 ms
Wall time: 894 ms


In [6]:
%%time
encoded_pixels = convert_predictions_to_rle_column(data)

CPU times: total: 6min 46s
Wall time: 6min 46s


### Формирование файла результатов в требуемом формате

In [7]:
submission_exampe = pd.read_csv('https://video.ittensive.com/machine-learning/clouds/sample_submission.csv.gz')

In [8]:
submission = pd.DataFrame(columns=submission_exampe.columns)
submission["EncodedPixels"] = encoded_pixels
submission["Image_Label"] = pd.Series([file_name + "_Fish" for file_name in test_files_list])
submission.head(10)

,Image_Label,EncodedPixels
0,002f507.jpg_Fish,None
1,0035ae9.jpg_Fish,None
2,0038327.jpg_Fish,None
3,004f759.jpg_Fish,None
4,005ba08.jpg_Fish,None
5,006440a.jpg_Fish,9423 2 31514 8 31525 17 31895 39 31935 2 32274...
6,006f61b.jpg_Fish,79615 2 79631 2 79999 2 80015 2 80383 2 80399 ...
7,0078e5a.jpg_Fish,36488 1 36872 1 37255 2 37639 2 37652 1 37655 ...
8,008fc8a.jpg_Fish,62624 1 62983 34 63367 36 63750 38 63789 2 641...
9,0096937.jpg_Fish,3583 1 3585 5 3958 3 3963 22 4338 36 4721 40 5...


In [9]:
submission.to_csv('submission.csv.gz')